# Colab-video-inference

My repo: [styler00dollar/Colab-video-inference](https://github.com/styler00dollar/Colab-video-inference)

A basic colab to take video models to create videos. Load a jit or pth. Currently only supporting batch_size 1.

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# get data
%cd /content
!wget 

In [ ]:
#@markdown adjust code if needed
from types import FrameType
from PIL import Image
import argparse
import torch
import torchvision.transforms as TF
import torch.nn as nn
import os
import numpy as np
import cv2
import warnings
import numpy
from tqdm import tqdm
import glob
warnings.filterwarnings("ignore")

torch.backends.cudnn.benchmark = True

input_video = "/content/m.webm" #@param

%cd /content/
!sudo rm -rf /content/png/
!mkdir /content/png/
%cd /content/
%shell ffmpeg -i {input_video} png/%05d.png

frames_dir = "/content/png/"
files = sorted(glob.glob(frames_dir + '/**/*.png', recursive=True))
del files[-1]

model_path = "/content/drive/MyDrive/CAIN/rvpv2/Checkpoint_4_55000_G.pt" #@param
# model, load it manually
#model = 
#model.load_state_dict(torch.load(model_path))

# load a jit
model = torch.jit.load(model_path)
model.cuda().eval()

input_frame = 1
for f in tqdm(files):
  with torch.no_grad():
    filename_frame_1 = f
    filename_frame_2 = os.path.join(frames_dir, f'{input_frame+1:0>5d}.png')
    output_frame_file_path = os.path.join(frames_dir, f"{input_frame:0>5d}_0.5.png")

    img1 = cv2.imread(filename_frame_1)
    img2 = cv2.imread(filename_frame_2)

    # resize input
    img1 = cv2.resize(img1, (1280, 720)) #, interpolation=cv2.INTER_NEAREST)
    img2 = cv2.resize(img2, (1280, 720)) #, interpolation=cv2.INTER_NEAREST)

    img1_new = cv2.resize(img1, (1280, 720)) #, interpolation=cv2.INTER_NEAREST)
    img2_new = cv2.resize(img2, (1280, 720)) #, interpolation=cv2.INTER_NEAREST)
    cv2.imwrite(filename_frame_1, img1_new)
    cv2.imwrite(filename_frame_2, img2_new)

    #img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2YUV)
    #img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2YUV)
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

    img1 = torch.from_numpy(img1).unsqueeze(0).permute(0,3,1,2)/255
    img2 = torch.from_numpy(img2).unsqueeze(0).permute(0,3,1,2)/255

    out = model(img1.cuda().contiguous(), img2.cuda().contiguous())

    # to numpy and save
    out = out.data.mul(255).mul(255 / 255).clamp(0, 255).round()
    out = out.squeeze(0).permute(1, 2, 0).cpu().numpy() #*255
    out = out.astype(np.uint8)
    #out = cv2.cvtColor(out, cv2.COLOR_YUV2RGB)
    out = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
    out = cv2.resize(out, (1280,720)) #, interpolation=cv2.INTER_NEAREST)

    cv2.imwrite(output_frame_file_path, out)

    input_frame += 1

%cd /content/png/
%shell ffmpeg -y -r 48 -f image2 -pattern_type glob -i '/content/png/*.png' -crf 18 /content/out.mp4
%cd /content/

In [ ]:
# create jit, assuming you defined the model prior
traced_model = torch.jit.trace(model, (torch.randn(1,3,720,1280).cuda(),torch.randn(1,3,720,1280).cuda()))
torch.jit.save(traced_model, "traced2.pt")